# **Notebook 4: Modeling & Evaluation**

## Objectives

* Fit and evaluate a regression model to predict prices on houses.

## Inputs

* outputs/datasets/collection/house_prices_records.csv
* Instructions on which features to use for data cleaning and feature engineering can be found in respective notebooks.

## Outputs

* Train set (features and target)
* Test set (features and target)
* Data cleaning and Feature Engineering pipeline
* Modeling pipeline
* Feature importance plot



---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [6]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/housing/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [7]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [8]:
current_dir = os.getcwd()
current_dir

'/workspaces/housing'

# Step 1: Load data

Here we load the dataset and separate the target variable ('y') from the predictor variables ('X').



In [9]:
import numpy as np
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/house_prices_records.csv"))

# Separate predictors and target
X = df.drop(['SalePrice'], axis=1)
y = df['SalePrice']

print(X.shape)
X.head(3)

(1460, 23)


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,8450,65.0,196.0,61,5,7,856,0.0,2003,2003
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,9600,80.0,0.0,0,8,6,1262,NaN,1976,1976
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,11250,68.0,162.0,42,5,7,920,NaN,2001,2002


---

# Step 2: ML Pipeline with all data

## ML pipeline for Data Cleaning and Feature Engineering

In [ ]:
from sklearn.pipeline import Pipeline

# Feature Engineering
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine.encoding import OrdinalEncoder


def PipelineDataCleaningAndFeatureEngineering():
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'])),
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None,
         method="spearman", threshold=0.6, selection_method="variance")),
    ])

    return pipeline_base


PipelineDataCleaningAndFeatureEngineering()


## ML Pipeline for Modelling and Hyperparameter Optimisation

In [ ]:
# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms for regression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor


# adapt the function to regression
def PipelineReg(model):
    pipeline_base = Pipeline([
        ("scaler", StandardScaler()),
        ("feat_selection", SelectFromModel(model)),
        ("model", model),
    ])

    return pipeline_base


Custom Class for Hyperparameter Optimisation

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")

            model = PipelineReg(self.models[key])  # changed this line to my own function from cell above
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, )
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        return df[columns], self.grid_searches

## Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=0,
)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Note: we do not handle target imbalance at this point since the target in this project is a continuous variable. It's possible to come back to this step later and add a cell to normalize the distribution of the target variable to fit the model better.

## Grid Search CV - Sklearn

Use standard hyperparameters to find most suitable algorithm

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

models_quick_search = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
    "SVR": SVR(),
}

params_quick_search = {
    "LinearRegression": {},
    "Ridge": {},
    "Lasso": {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
    "SVR": {},
}

And then Quick GridSearch CV

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error

# We want to minimize MSE, hence greater_is_better=False.
scorer = make_scorer(mean_squared_error, greater_is_better=False)

search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring=scorer, n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary 

### Do an extensive search on the most suitable algorithm to find the best hyperparameter configuration.

Define model and parameters, for Extensive Search

In [ ]:
models_search = {
    "XGBRegressor": XGBRegressor(random_state=0),
}

# documentation to help on hyperparameter list: 
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

params_search = {
    "XGBRegressor": {
        'model__learning_rate': [1e-1, 1e-2, 1e-3], 
        'model__max_depth': [3, 10, None],
        'model__n_estimators': [50, 100, 200],  # add more parameters for tuning
    },
}

Extensive GridSearch CV

In [ ]:
from sklearn.metrics import mean_squared_error, make_scorer

# We want to minimize MSE, hence greater_is_better=False.
scorer = make_scorer(mean_squared_error, greater_is_better=False)

search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring=scorer, n_jobs=-1, cv=5)

Check results

Here we get a summary table showing the results from the GridSearchCV operation. It includes the mean_score, which we can use to compare the performance of different hyperparameters configurations. The one with the highest (or lowest, if your metric is an error/loss to be minimized) mean_score would be the optimal choice.

Keep in mind that the score here is the mean_squared_error with greater_is_better=False, so the model configuration with the most negative mean score is the best one, as it signifies the lowest mean squared error.

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary 

Get best model name programmatically

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

Parameters for best model. This contain a dictionary of the optimal hyperparameters for the best model as determined by the GridSearchCV. It doesn't contain the model itself, but only the parameters that gave the best performance during the cross-validation grid search

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

Get the complete pipeline of the best model, including all preprocessing steps and the estimator itself, with the parameters set to their optimal values:

In [ ]:
pipeline_rgr = grid_search_pipelines[best_model].best_estimator_
pipeline_rgr

In other words, pipeline_rgr is ready for immediate use on data, while best_parameters would need to be used to set the parameters of a new instance of the model before fitting it to the data.

## Assess feature importance

In [ ]:
X_train.head(3)

In [ ]:
# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': X_train.columns[pipeline_rgr['feat_selection'].get_support()],
    'Importance': pipeline_rgr['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# re-assign best_features order
best_features = df_feature_importance['Feature'].to_list()

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

Note to self: This should create a bar plot of feature importances, showing the most important features on the top.

Please note that you'll need to be sure that your pipeline_rgr model's estimator ('model' step in your pipeline) indeed has a .feature_importances_ attribute. The XGBoost, RandomForest, and GradientBoosting regressors do, but others like SVMs and linear models do not. In the latter case, this cell will fail.

Please be careful when interpreting these feature importances - they reflect the model's internal working, but they might not directly correspond to "importance" in a business or logical sense. They also don't indicate the direction of influence a feature has (whether high feature value increases or decreases the predicted target), only the magnitude of that influence.

## Evaluate Pipeline on Train and Test Sets

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def regression_metrics(X, y, pipeline):
    prediction = pipeline.predict(X)

    print('---  Regression Metrics  ---')
    print(f"Mean Absolute Error (MAE): {mean_absolute_error(y, prediction)}")
    print(f"Mean Squared Error (MSE): {mean_squared_error(y, prediction)}")
    print(f"Root Mean Squared Error (RMSE): {mean_squared_error(y, prediction, squared=False)}")
    print(f"R-squared (R^2): {r2_score(y, prediction)}")

def reg_performance(X_train, y_train, X_test, y_test, pipeline):
    print("#### Train Set #### \n")
    regression_metrics(X_train, y_train, pipeline)

    print("\n#### Test Set ####\n")
    regression_metrics(X_test, y_test, pipeline)


This version of the function above calculates and prints out regression metrics, which are more relevant for evaluating how well a regression model performs. These include MAE, MSE, RMSE, and R^2.

After running this function, we should see these metrics printed out for both our training set and test set. This will give us an idea of how well our model is doing on both the data it was trained on and on new, unseen data.

In [ ]:
reg_performance(X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                pipeline=pipeline_rgr)

# The function reg_performance will print out the regression metrics 
# for our training and test datasets using the pipeline we trained.

# Step 3: Refit pipeline with best features

# Next step: Notebook 2

In notebook 2 we will move on the the next next step in our data analysis process: Exploratory Data Analysis (EDA).